In [1]:
import os
import numpy as np
import torch
import clip
import json as js
import numpy as np
from natsort import natsorted
import pandas as pd
from typing import List
from PIL import Image, ImageFont, ImageDraw

In [2]:
MODEL = "ViT-B/32"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

METADATA_PATH = "../data/metadata/"
KEYFRAME_PATH = "../data/keyframes/"
FEATURE_PATH = "../data/features/"
MAP_KEYFRAMES = "../data/map-keyframes/"
VIDEOS_PATH = "../data/videos/"
SCRIPT_PATH = "../data/scripts/"

WORKSPACE = "./vectordb"

cuda


In [3]:
class ImageEmbedding:
    def __init__(self):
        self.device = DEVICE
        self.model, self.preprocess = clip.load(MODEL, device=self.device)

    def __call__(self, img_path: str) -> np.ndarray:
        image = self.preprocess(Image.open(img_path)).unsqueeze(0).to(self.device)
        with torch.no_grad():
            text_feature = self.model.encode_image(image)[0]
        return text_feature.detach().cpu().numpy()


In [4]:
img_emd = ImageEmbedding()

In [5]:
path = "../data/keyframes/L01_V001/0001.jpg"
feat1 = img_emd(path)

In [6]:
path = "../data/keyframes/L01_V001/0002.jpg"
for i in range(1000):
    feat2 = img_emd(path)

KeyboardInterrupt: 

In [ ]:
print(np.vstack([feat1, feat2]).shape)

(2, 512)


In [8]:
video_name = "L01_V001"
map_kf = pd.read_csv(
                os.path.join(MAP_KEYFRAMES, video_name + ".csv")
            )
metadata = js.load(open(os.path.join(METADATA_PATH, video_name + ".json")))
print(map_kf)
print(metadata)
print(metadata["length"] * map_kf["fps"][0])

       n  pts_time   fps  frame_idx
0      1      0.00  25.0          0
1      2      4.00  25.0        100
2      3     10.84  25.0        271
3      4     13.40  25.0        335
4      5     13.84  25.0        346
..   ...       ...   ...        ...
231  232   1082.00  25.0      27050
232  233   1086.24  25.0      27156
233  234   1093.24  25.0      27331
234  235   1099.88  25.0      27497
235  236   1104.40  25.0      27610

[236 rows x 4 columns]
{'author': '60 Giây Official', 'channel_id': 'UCRjzfa1E0gA50lvDQipbDMg', 'channel_url': 'https://www.youtube.com/channel/UCRjzfa1E0gA50lvDQipbDMg', 'description': None, 'keywords': [], 'length': 1109, 'publish_date': '01/12/2022', 'thumbnail_url': 'https://i.ytimg.com/vi/HNsRpkryGXA/hqdefault.jpg?sqp=-oaymwEXCJADEOABSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDT_1e4WaAw3ZFe1c31UF2n23_5Kw', 'title': '60 Giây Sáng - Ngày 01/12/2022 - HTV Tin Tức Mới Nhất', 'watch_url': 'https://youtube.com/watch?v=HNsRpkryGXA'}
27725.0


In [10]:
import cv2
cap = cv2.VideoCapture(os.path.join(VIDEOS_PATH, video_name + ".mp4"))



True

In [11]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 27156)
res, frame = cap.read()
cv2.imwrite(video_name+'_frame_'+str(27156)+'.jpg',res)

True